# Text Tokenizer

자연어처리에 있어 토크나이저는 문장을 작은 단위의 토큰(단어 또는 부분단어)으로 분할하는 역할을 하며, 
본 실습에서는 대표적인 SubWord Tokenizer 방법들에 대해 살펴보도록 하겠습니다.  
- BPE Tokenization (Algorithm) 
- Pretrained Tokenizer (WordPiece)  
- Tokenizer Training 

## 0. Setup

In [1]:
# MLP Suwon 설정 필요
import os

os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'
os.environ['HTTP_PROXY'] ='http://75.17.107.42:8080'
os.environ['HTTPS_PROXY'] ='http://75.17.107.42:8080'

In [2]:
# MLP Suwon 설정 필요
import ssl

if hasattr(ssl, '_create_unverified_context'):
   ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# !pip install --user transformers==4.38.2

## 1. BPE Tokenization (Algorithm)

BPE 토크나이저는 기본 단위(단일 문자)의 리스트로 시작해서 가장 빈번하게 함께 등장하는 토큰을 합쳐 어휘사전에 새롭게 추가하는 방식으로 원하는 어휘사전 크기에 도달할 때까지 점진적으로 새 토큰 만드는 과정을 반복합니다.  
아래 실습에서는 논문(https://arxiv.org/pdf/1508.07909.pdf) 에 언급된 알고리즘을 그대로 구현해 보겠습니다.

In [3]:
import re, collections
from IPython.display import display, Markdown, Latex

In [4]:
num_merges = 10

In [5]:
vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>' : 6,
         'w i d e s t </w>' : 3
         }

In [6]:
def get_stats(vocab):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('Frequences :', dict(pairs))
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("New merge: {}".format(best))
    print("Vocabulary: {}".format(vocab))

### Iteration 1

Frequences : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
New merge: ('e', 's')
Vocabulary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

Frequences : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
New merge: ('es', 't')
Vocabulary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

Frequences : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('new', 'est</w>')
Vocabulary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

Frequences : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('low', '</w>')
Vocabulary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

Frequences : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('w', 'i')
Vocabulary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


## 2. Pretrained Tokenizer (WordPiece)

트랜스포머스는 사전 훈련된 언어모델을 활용하기 위하여 언어모델 구축에 사용된 토크나이저를 로딩하는 **AutoTokenizer** 클래스를 
제공합니다.  
이 클래스의 **.from_pretrained()** 메소드를 이용하여 모델ID나 로컬 파일 경로로 호출하면 됩니다.  
대표적인 Sub-word Tokenizer 방식으로 WordPiece 계열의 **BERT Tokenizer**를 이용하여 실습해 보겠습니다.

In [7]:
from transformers import AutoTokenizer

# [실습] 다음 코드를 완성하세요!!
# 'distilbert-base-uncased' 모델에 사용된 토크나이저를 불러옵니다.
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer.tokenize("Time flies like an arrow!")

['time', 'flies', 'like', 'an', 'arrow', '!']

In [8]:
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)
print(tokenizer.model_input_names)

30522
512
['input_ids', 'attention_mask']


In [9]:
sentence = "Tokenizing a text is splitting it into words or subwords."

# [실습] 다음 코드를 완성하세요!!
# tokenizer를 이용하여 sentence를 토크나이즈합니다.
tokenized_sentence = tokenizer.tokenize(sentence)
print(tokenized_sentence)

['token', '##izing', 'a', 'text', 'is', 'splitting', 'it', 'into', 'words', 'or', 'sub', '##words', '.']


In [10]:
inputs = tokenizer(sentence)

encoded_sentence = inputs["input_ids"]
print(encoded_sentence)

[101, 19204, 6026, 1037, 3793, 2003, 14541, 2009, 2046, 2616, 2030, 4942, 22104, 1012, 102]


In [11]:
# [실습] 다음 코드를 완성하세요!!
# tokenizer를 이용하여 encoded_sentence를 문장으로 복원합니다.
decoded_sentence = tokenizer.decode(encoded_sentence)
print(decoded_sentence)

[CLS] tokenizing a text is splitting it into words or subwords. [SEP]


In [12]:
sentence_a = "Time flies like an arrow."
sentence_b = "This sentence is at least longer than the sentence A."

padded_sentence = tokenizer([sentence_a, sentence_b], padding=True)
print(padded_sentence["input_ids"])

[[101, 2051, 10029, 2066, 2019, 8612, 1012, 102, 0, 0, 0, 0, 0], [101, 2023, 6251, 2003, 2012, 2560, 2936, 2084, 1996, 6251, 1037, 1012, 102]]


In [13]:
padded_sentence["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

## 3. Tokenizer Training

특정 언어 모델이 당신이 원하는 언어를 제대로 지원하지 않거나, 현재 보유한 코퍼스가 언어 모델과 특성이 다른 경우, 해당 코퍼스에 적합하게 적응된 토크나이저를 이용하여 모델을 새롭게 학습하기를 원할 수도 있습니다. 이를 위해서는 그 데이터셋에 대한 새로운 토크나이저를 학습해야 합니다. 예로, 파이썬 코드를 토큰화 하기 위해 BPE 토크나이저(GPT2-XL)를 재훈련해 보도록 하겠습니다.  

In [14]:
from transformers import AutoTokenizer

model_ckpt = "gpt2-xl"

old_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [15]:
print(f"Vocabulary Size: {len(old_tokenizer)}")

Vocabulary Size: 50257


Python Code 이용하여 GPT2-XL 토크나이저를 테스트해 보겠습니다.  
'Ġ', 'Ċ' 와 같은 문자가 있는 것은 GPT-2 토크나이저가 유니코드 문자가 아닌 바이트 단위로 동작하기 때문입니다.
('Ġ': Space, 'Ċ': Line Feed,...)

In [16]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [17]:
import keyword

for keyw in keyword.kwlist:
    if keyw not in old_tokenizer.vocab:
        print(f'Keyword `{keyw}` does not exist in Vocabulary!')

Keyword `async` does not exist in Vocabulary!
Keyword `await` does not exist in Vocabulary!
Keyword `elif` does not exist in Vocabulary!
Keyword `finally` does not exist in Vocabulary!
Keyword `nonlocal` does not exist in Vocabulary!
Keyword `yield` does not exist in Vocabulary!


파이썬 코드 데이터(code_search_net/python)을 이용하여 토크나이저를 새로 학습하겠습니다.

데이터셋 **"code search_net"** 는 CodeSearchNet Challenge를 위해 구축된 것으로, GitHub 상의 오픈소스 코드를 포함하고 있으며 파이썬을 포함 몇개 프로그래밍 언어로 구분되어 있습니다.

In [18]:
"""
The repository for code_search_net contains custom code which must be executed to correctly load the dataset. 
You can inspect the repository content at code_search_net.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
"""

from datasets import load_dataset

raw_datasets = load_dataset("code_search_net", "python", trust_remote_code=True)

In [19]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [20]:
print(raw_datasets["train"][123456]["whole_func_string"])

def _initialize_monitoring_services_queue(self, chain_state: ChainState):
        """Send the monitoring requests for all current balance proofs.

        Note:
            The node must always send the *received* balance proof to the
            monitoring service, *before* sending its own locked transfer
            forward. If the monitoring service is updated after, then the
            following can happen:

            For a transfer A-B-C where this node is B

            - B receives T1 from A and processes it
            - B forwards its T2 to C
            * B crashes (the monitoring service is not updated)

            For the above scenario, the monitoring service would not have the
            latest balance proof received by B from A available with the lock
            for T1, but C would. If the channel B-C is closed and B does not
            come back online in time, the funds for the lock L1 can be lost.

            During restarts the rationale from above has to be 

In [21]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

In [22]:
training_corpus = get_training_corpus()

기존 토크나이저와 동일한 특성을 가진 새로운 토크나이저를 학습하기 위해 AutoTokenizer.train_new_from_iterator()를 사용하겠습니다.

- 목표하는 Vocabulary Size 설정
- 토크나이저 학습을 위해 입력 문자열을 공급하는 Iterator 설정
- train_new_from_iterator() 메소드 호출

토크나이저 학습에 약 2~3분 정도 소요됩니다.

In [23]:
# [실습] 다음 코드를 완성하세요!!
# training_corpus를 이용하여 토크나이저를 학습합니다.
new_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [24]:
tokens = new_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [25]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [26]:
import keyword

for keyw in keyword.kwlist:
    if keyw not in new_tokenizer.vocab:
        print(f'Keyword `{keyw}` does not exist in Vocabulary!')

Keyword `nonlocal` does not exist in Vocabulary!


- Ref. https://huggingface.co/learn/nlp-course/chapter6